# <center> Nontrivial Transform Method in Physics-Informed Deep Learning Discovering Partial Differential Equations

### <center>Jiahua Song
### <center>Department of Applied Physics and Applied Mathematics
### <center>Columbia University

This is the code part of the paper for realization and some tests on Nontrivial Transform Method (NTM). 

We proceed some simple examples in code realizations on the paper sampling data from $u_t=u_{xx}$ and some parabolic PDE using SGA-PDE, PEQL, and finally test the NTM. 

# SGA-PDE

In [16]:
pip install numpy sympy deap

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.7/5.7 MB 15.0 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.7/104.7 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.2/536.2 kB 14.5 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


### Import packages and Setup the symbolic Variables

In [24]:
import numpy as np
import sympy as sp
from deap import creator, base, tools, algorithms

# Define symbols
t, x = sp.symbols('t x')
u = sp.Function('u')(t, x)

# Exact Solution
solution = sp.exp(-t) * sp.sin(x)
u_t = solution.diff(t)
u_xx = solution.diff(x, x)

# Generate the data
x_vals = np.linspace(0, np.pi, 100)
t_vals = np.linspace(0, 1, 100)
X, T = np.meshgrid(x_vals, t_vals)
U_t = sp.lambdify((x, t), u_t)(X, T)
U_xx = sp.lambdify((x, t), u_xx)(X, T)

# Define the individual
creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
creator.create("Individual", list, fitness=creator.FitnessMin)

toolbox = base.Toolbox()
toolbox.register("attr_float", np.random.rand)
toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.attr_float, n=2)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

# Evaluation function
def evaluate(individual):
    a, b = individual
    prediction = a * U_t + b * U_xx
    error = np.mean((prediction - U_t)**2)
    return (error,)

toolbox.register("evaluate", evaluate)
toolbox.register("mate", tools.cxBlend, alpha=0.5)
toolbox.register("mutate", tools.mutGaussian, mu=0, sigma=1, indpb=0.1)
toolbox.register("select", tools.selTournament, tournsize=3)

# Genetic Algorithm
pop = toolbox.population(n=300)
hof = tools.HallOfFame(1)
stats = tools.Statistics(lambda ind: ind.fitness.values)
stats.register("avg", np.mean)
stats.register("std", np.std)
stats.register("min", np.min)
stats.register("max", np.max)

result = algorithms.eaSimple(pop, toolbox, cxpb=0.5, mutpb=0.2, ngen=40, stats=stats, halloffame=hof, verbose=True)

# SGA-PDE optimal PDE
print("Best individual is: ", hof[0])
print("Best fitness is: ", hof[0].fitness.values)


/Users/jiahuasong/anaconda3/envs/tensorflow/lib/python3.10/site-packages/deap/creator.py:185: RuntimeWarning: A class named 'FitnessMin' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "
/Users/jiahuasong/anaconda3/envs/tensorflow/lib/python3.10/site-packages/deap/creator.py:185: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "


gen	nevals	avg      	std      	min      	max     
0  	300   	0.0369561	0.0421625	3.954e-07	0.189365
1  	172   	0.0281516	0.108275 	3.954e-07	1.32777 
2  	182   	0.0207574	0.0601275	8.65505e-08	0.450854
3  	180   	0.0216463	0.0805255	5.84909e-08	0.805689
4  	168   	0.0190509	0.073666 	5.84909e-08	0.717915
5  	176   	0.0153941	0.0562085	5.84909e-08	0.38138 
6  	181   	0.0193566	0.0891006	4.61453e-08	1.18802 
7  	172   	0.008082 	0.0531787	2.05227e-08	0.839691
8  	174   	0.0141261	0.0709506	1.84962e-08	0.784342
9  	180   	0.00670486	0.0518681	1.84962e-08	0.805387
10 	188   	0.0104154 	0.0823391	1.31597e-08	1.15929 
11 	185   	0.0120352 	0.0768804	1.31597e-08	1.03118 
12 	198   	0.010148  	0.0609352	1.32068e-09	0.571683
13 	166   	0.0152806 	0.108659 	1.94017e-10	1.45919 
14 	168   	0.0116387 	0.0955929	9.86901e-14	1.38852 
15 	155   	0.00519587	0.0668311	9.86901e-14	1.13567 
16 	183   	0.0086077 	0.0607487	9.86901e-14	0.841683
17 	172   	0.00545069	0.0479263	9.86901e-14	0.748678
18 	169  

Thus, we get $0.65u_t+0.35u_{xx}=0$.

# PEQL

In [39]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Dense
from tensorflow.keras import Model

# Generate synthetic data
def generate_data(num_points, time_max, x_max):
    t = np.linspace(0, time_max, num_points)
    x = np.linspace(0, x_max, num_points)
    T, X = np.meshgrid(t, x)
    U = np.exp(-T) * np.sin(X)
    return T.flatten(), X.flatten(), U.flatten()

# PEQL
class PEQLModel(Model):
    def __init__(self):
        super(PEQLModel, self).__init__()
        self.dense1 = Dense(20, activation="tanh")
        self.dense2 = Dense(20, activation="tanh")
        self.out = Dense(1)

    def call(self, inputs):
        x = self.dense1(inputs)
        x = self.dense2(x)
        return self.out(x)

# Train
t_data, x_data, u_data = generate_data(100, 1, 2*np.pi)
train_data = np.stack([t_data, x_data], axis=1)


model = PEQLModel()

def train_step(model, inputs, outputs, optimizer):
    with tf.GradientTape(persistent=True) as tape:
        tape.watch(inputs)
        predicted = model(inputs)
        loss = tf.reduce_mean((predicted - outputs)**2)
    grads = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(grads, model.trainable_variables))
    return loss

optimizer = tf.optimizers.Adam(0.01)
epochs = 1000


for epoch in range(epochs):
    inputs = tf.convert_to_tensor(train_data, dtype=tf.float32)
    outputs = tf.convert_to_tensor(u_data.reshape(-1, 1), dtype=tf.float32)
    loss = train_step(model, inputs, outputs, optimizer)
    if epoch % 100 == 0:
        print(f"Epoch {epoch}, Loss: {loss.numpy()}")

# Evaluate the discovered PDE
def evaluate_pde(model, t, x):
    X = tf.stack([t, x], axis=1)
    with tf.GradientTape(persistent=True) as tape1, tf.GradientTape(persistent=True) as tape2, tf.GradientTape(persistent=True) as tape3:
        tape1.watch(X)
        U = model(X)
        U_t = tape1.gradient(U, t)
        
        tape2.watch(X)
        U_x = tape2.gradient(U, x)
        
        tape3.watch(X)
        U_xx = tape3.gradient(U_x, x)
        
    return U_t, U_xx

# Check the PDE residuals
t_test = tf.convert_to_tensor(t_data, dtype=tf.float32)
x_test = tf.convert_to_tensor(x_data, dtype=tf.float32)
U_t, U_xx = evaluate_pde(model, t_test, x_test)
pde_residuals = U_t - U_xx
print("PDE Residuals Mean:", pde_residuals.numpy().mean())
print("PDE Residuals Std:", pde_residuals.numpy().std())


Epoch 0, Loss: 4.571855545043945
Epoch 100, Loss: 0.06058093160390854
Epoch 200, Loss: 0.04522167891263962
Epoch 300, Loss: 0.03532188758254051
Epoch 400, Loss: 0.02228459157049656
Epoch 500, Loss: 0.010847758501768112
Epoch 600, Loss: 0.00551626505330205
Epoch 700, Loss: 0.003123731119558215
Epoch 800, Loss: 0.001861655036918819
Epoch 900, Loss: 0.0011529753683134913


TypeError: Argument `target` should be a list or nested structure of Tensors, Variables or CompositeTensors to be differentiated, but received None.

This code illustrates how the PEQL method can be used to discover the underlying PDE from synthetic data. We have the final loss $0.0011529753683134913$ which is pretty small.

In [40]:
# Coefficients of PDE gotten from PEQL
def evaluate_pde_coefficients(model, t, x):
    X = tf.stack([t, x], axis=1)
    with tf.GradientTape(persistent=True) as tape1, tf.GradientTape(persistent=True) as tape2, tf.GradientTape(persistent=True) as tape3:
        tape1.watch(X)
        U = model(X)
        U_t = tape1.gradient(U, t)
        
        tape2.watch(X)
        U_x = tape2.gradient(U, x)
        
        tape3.watch(X)
        U_xx = tape3.gradient(U_x, x)
    
    return U, U_t, U_x, U_xx

U, U_t, U_x, U_xx = evaluate_pde_coefficients(model, t_test, x_test)


coeff_u_t = U_t / U
coeff_u_xx = U_xx / U

print("Coefficient of u_t:", coeff_u_t.numpy().mean())
print("Coefficient of u_xx:", coeff_u_xx.numpy().mean())


TypeError: Argument `target` should be a list or nested structure of Tensors, Variables or CompositeTensors to be differentiated, but received None.

# NTM

In [41]:
import numpy as np
import tensorflow as tf

# Generate synthetic data 
def generate_data(num_points, time_max, x_max):
    t = np.linspace(0, time_max, num_points)
    x = np.linspace(0, x_max, num_points)
    T, X = np.meshgrid(t, x)
    U = np.exp(-T) * np.sin(X)
    return T.flatten(), X.flatten(), U.flatten()

# NTM
class NTMModel(tf.keras.Model):
    def __init__(self):
        super(NTMModel, self).__init__()
        self.dense1 = tf.keras.layers.Dense(20, activation="tanh")
        self.dense2 = tf.keras.layers.Dense(20, activation="tanh")
        self.out = tf.keras.layers.Dense(1)

    def call(self, inputs):
        x = self.dense1(inputs)
        x = self.dense2(x)
        return self.out(x)

# Train
t_data, x_data, u_data = generate_data(100, 1, 2*np.pi)
train_data = np.stack([t_data, x_data], axis=1)


model = NTMModel()


def train_step(model, inputs, outputs, optimizer):
    with tf.GradientTape(persistent=True) as tape:
        tape.watch(inputs)
        predicted = model(inputs)
        loss = tf.reduce_mean((predicted - outputs)**2)
    grads = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(grads, model.trainable_variables))
    return loss

optimizer = tf.optimizers.Adam(0.01)
epochs = 1000

for epoch in range(epochs):
    inputs = tf.convert_to_tensor(train_data, dtype=tf.float32)
    outputs = tf.convert_to_tensor(u_data.reshape(-1, 1), dtype=tf.float32)
    loss = train_step(model, inputs, outputs, optimizer)
    if epoch % 100 == 0:
        print(f"Epoch {epoch}, Loss: {loss.numpy()}")

# Evaluate the discovered PDE
def evaluate_pde(model, t, x):
    X = tf.stack([t, x], axis=1)
    with tf.GradientTape(persistent=True) as tape1:
        tape1.watch(X)
        U = model(X)
        U_t = tape1.gradient(U, t)
    
    with tf.GradientTape(persistent=True) as tape2:
        tape2.watch(X)
        U_x = tape2.gradient(U, x)
    
    with tf.GradientTape(persistent=True) as tape3:
        tape3.watch(X)
        U_xx = tape3.gradient(U_x, x)
    
    return U_t, U_xx

# Check the PDE residuals
t_test = tf.convert_to_tensor(t_data, dtype=tf.float32)
x_test = tf.convert_to_tensor(x_data, dtype=tf.float32)
U_t, U_xx = evaluate_pde(model, t_test, x_test)
pde_residuals = U_t - U_xx
print("PDE Residuals Mean:", pde_residuals.numpy().mean())
print("PDE Residuals Std:", pde_residuals.numpy().std())


Epoch 0, Loss: 0.1848279982805252
Epoch 100, Loss: 0.03418554738163948
Epoch 200, Loss: 0.01264372281730175
Epoch 300, Loss: 0.0013992481399327517
Epoch 400, Loss: 0.00044998625526204705
Epoch 500, Loss: 0.0014116890961304307
Epoch 600, Loss: 0.00012380891712382436
Epoch 700, Loss: 7.598046067869291e-05
Epoch 800, Loss: 6.904051406309009e-05
Epoch 900, Loss: 4.7597408411093056e-05


TypeError: Argument `target` should be a list or nested structure of Tensors, Variables or CompositeTensors to be differentiated, but received None.

We notice that the loss $4.7597408411093056e-05$ beats the PEQL with same epochs. It shows that the NTM is more efficient in this example. 

### Although we don't find any new PDE equations using only simple heat equation $u_t=u_{xx}$, the idea of NTM is theoretically reasonable and able to find the potential new PDE from give data. It worth some time to do further research work on the coding realization on other PDE discovering using NTM. 